## Imports

In [14]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

## I. Data-Cleaning

### 1. diet.csv

In [2]:
users_diet = pd.read_csv('../aufgabe/training_dataset/diet.csv')
# print(users_diet.head())
# users_diet.shape
# users_diet.dtypes

# Nullwerte
# users_diet.isnull().sum()

# 1. Spalte "Diet"
# -> ein Nullwert drin 
#users_diet[users_diet["Diet"].isna()]

# Author 646062A ohne Wert für Diet -> Weg mit dem Hund
users_diet = users_diet.drop(users_diet[users_diet["Diet"].isna()].index).reset_index(drop=True)

# Weitere missing values finden: z.B. <empty field>, "0", ".", "999", "NA" ...
# users_diet[(users_diet["Diet"] == "") | (users_diet["Diet"] == ".") | (users_diet["Diet"] == "999")]


# 2. Spalte "Age"
# users_diet[users_diet["Age"] > 100]
# users_diet[users_diet["Age"] < 5]
# print("Range Alter", users_diet["Age"].min(), users_diet["Age"].max())

# 3. Spalte "AuthorId"
# print("Einzigartige IDs: ", users_diet["AuthorId"].nunique(), users_diet.shape[0])


# Datentyp bei "Diet" zu Category ändern
users_diet["Diet"] = users_diet['Diet'].astype("category")
users_diet.dtypes


AuthorId      object
Diet        category
Age            int64
dtype: object

### 2. reviews.csv

In [47]:
reviews_data = pd.read_csv("../aufgabe/training_dataset/reviews.csv")

# Test-Daten -> TestSetId != NaN, die anderen nicht für Modelle verwenden
reviews_data_test = reviews_data[reviews_data["TestSetId"].isna()].reset_index(drop=True)
reviews_data_test.loc[reviews_data_test['Rating'].isna(), "Rating"] = 999
reviews_data_test["Rating"] = reviews_data_test["Rating"].astype("category")

# Like column zu boolean
reviews_data_test["Like"] = reviews_data_test["Like"].astype("bool")

# reviews_data_test.dtypes


/var/folders/j0/z37d__mj75g9k_jcd30hbw0c0000gn/T/ipykernel_1499/2135823843.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews_data = pd.read_csv("../aufgabe/training_dataset/reviews.csv")


,AuthorId,RecipeId,Rating,Like,TestSetId


### 3. requests.csv

In [99]:
requests_data = pd.read_csv("../aufgabe/training_dataset/requests.csv")
requests_data.head()

# Column: Time
# Runden
""" 
Column Time -> runden, da Nachkommastellen bei Kochzeit irrelevant
Beschreibung: The duration a recipe should take at most (including the time reserved
for the preparation and cooking).
"""
requests_data["Time"] = requests_data["Time"].round(1)

# Teilweise negative Werte -> 0
requests_data.loc[requests_data["Time"] <= 0, "Time"] = 0
"""
Test, ob negative Werte immer False als "Like" haben -> stimmt aber nicht siehe Code:
Requests mit [AuthorId, RecipeId] time <= 0 mit reviews [AuthorId, RecipeId] joinen und dort like checken
joined_data = requests_data.merge(reviews_data_test, on=["AuthorId", "RecipeId"], how="left")
joined_data = joined_data[joined_data["Time"] <= 0]
joined_data = joined_data[~joined_data["Like"].isna()]
"""

# Column: HighCalories
requests_data["HighCalories"] = requests_data["HighCalories"].astype("bool")

# Column: HighProtein
"""
2 Werte: Indifferent und Yes
Daraus wird boolean indifferent = False und Yes = True
"""
requests_data.loc[requests_data["HighProtein"] == "Indifferent", "HighProtein"] = 0
requests_data.loc[requests_data["HighProtein"] == "Yes", "HighProtein"] = 1
requests_data["HighProtein"] = requests_data["HighProtein"].astype("bool")

# Column: LowFat
requests_data["LowFat"] = requests_data["LowFat"].astype("bool")

# Column: LowSugar
"""
2 Werte: Indifferent und 0. Interpretation: 0 -> user braucht kein low-sugar Inhalt, Indifferent -> User ist es egal
Daraus wird boolean 0 = False und indifferent = True
"""
requests_data.loc[requests_data["LowSugar"] == "0", "LowSugar"] = 0
requests_data.loc[requests_data["LowSugar"] == "Indifferent", "LowSugar"] = 1
requests_data["LowSugar"] = requests_data["LowSugar"].astype("bool")

# Column HighFiber
requests_data["HighFiber"] = requests_data["HighFiber"].astype("bool")

# requests_data["HighFiber"].unique()
# requests_data.head(20)


,AuthorId,RecipeId,Time,HighCalories,HighProtein,LowFat,LowSugar,HighFiber
0,2001012259B,73440,1800.0,False,False,False,False,False
1,437641B,365718,4201.8,False,True,False,True,True
2,1803340263D,141757,6299.9,False,False,True,True,False
3,854048B,280351,19801.4,False,True,True,False,True
4,2277685E,180505,5400.1,False,False,False,False,False
5,2002463647E,350271,1500.7,False,False,False,False,False
6,181611B,21518,3901.5,True,False,False,True,True
7,63028B,137143,2699.8,False,False,False,True,False
8,84090D,211563,4799.0,True,True,True,False,True
9,935221A,29280,299.9,False,False,False,False,True
